# You Only Look Once: Unified, Real-Time Object Detection  
- [paper review](https://arxiv.org/pdf/1506.02640.pdf)

<img src = "https://github.com/fned49/blog/blob/main/YOLO/figure/figure1.png?raw=true" width=600>  